In [19]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

pic_dataset = load_dataset('PiC/phrase_similarity', 'PS-hard')
pic_df = pd.DataFrame(pic_dataset['train'])

corpus = pic_df['sentence1'].tolist() + pic_df['sentence2'].tolist()
word2vec_model = Word2Vec(sentences=[sentence.split() for sentence in corpus], vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

pic_df['vector_sentence1'] = pic_df['sentence1'].apply(lambda x: get_sentence_vector(x, word2vec_model))
pic_df['vector_sentence2'] = pic_df['sentence2'].apply(lambda x: get_sentence_vector(x, word2vec_model))

pic_df = pic_df.dropna(subset=['vector_sentence1', 'vector_sentence2'])
pic_df['cosine_similarity'] = pic_df.apply(lambda row: cosine_similarity([row['vector_sentence1']], [row['vector_sentence2']])[0][0], axis=1)
print(pic_df[['sentence1', 'sentence2', 'cosine_similarity']])


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset phrase_similarity downloaded and prepared to C:/Users/sanch/.cache/huggingface/datasets/PiC___phrase_similarity/PS-hard/2.0.1/7edf99614bd0c5125d3fc3b7d592122a988f23efc1e72dd1e01759bc68359302. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

                                              sentence1  \
0     newly formed camp is released from the membran...   
1     According to one data, in 1910, on others – in...   
2     Note that Fact 1 does not assume any particula...   
3     Assessment-Center are usually group-processes ...   
4     At the end of the 1980s, a different cross had...   
...                                                 ...   
6999  Nereus and Achilleus a similar notice was admi...   
7000  Color quantization is the process of creating ...   
7001  petra is a dutch tool with which patients and ...   
7002  However, granting this access, especially to a...   
7003  The show was immediately picked up for a full ...   

                                              sentence2  cosine_similarity  
0     recently made encampment is released from the ...           0.999982  
1     According to a particular statistic, in 1910, ...           0.999819  
2     Note that Fact 1 does not assume any specific ...     

In [21]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
pic_dataset = load_dataset('PiC/phrase_similarity', 'PS-hard')

pic_df = pd.DataFrame(pic_dataset['train'])

corpus = pic_df['sentence1'].tolist() + pic_df['sentence2'].tolist()
word2vec_model = Word2Vec(sentences=[sentence.split() for sentence in corpus], vector_size=100, window=5, min_count=1, workers=4)
def get_sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

pic_df['vector_sentence1'] = pic_df['sentence1'].apply(lambda x: get_sentence_vector(x, word2vec_model))
pic_df['vector_sentence2'] = pic_df['sentence2'].apply(lambda x: get_sentence_vector(x, word2vec_model))
pic_df = pic_df.dropna(subset=['vector_sentence1', 'vector_sentence2'])

pic_df['cosine_similarity'] = pic_df.apply(lambda row: cosine_similarity([row['vector_sentence1']], [row['vector_sentence2']])[0][0], axis=1)

X = pic_df[['cosine_similarity']]
y = pic_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Found cached dataset phrase_similarity (C:/Users/sanch/.cache/huggingface/datasets/PiC___phrase_similarity/PS-hard/2.0.1/7edf99614bd0c5125d3fc3b7d592122a988f23efc1e72dd1e01759bc68359302)


  0%|          | 0/3 [00:00<?, ?it/s]

Accuracy: 0.4989293361884368


In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

pic_dataset = load_dataset('PiC/phrase_similarity', 'PS-hard')

pic_df = pd.DataFrame(pic_dataset['train'])
label_encoder = LabelEncoder()
pic_df['label'] = label_encoder.fit_transform(pic_df['label'])
train_df, test_df = train_test_split(pic_df, test_size=0.2, random_state=42)
corpus = pic_df['sentence1'].tolist() + pic_df['sentence2'].tolist()
word2vec_model = Word2Vec(sentences=[sentence.split() for sentence in corpus], vector_size=100, window=5, min_count=1, workers=4)
def get_sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

train_df['vector_sentence1'] = train_df['sentence1'].apply(lambda x: get_sentence_vector(x, word2vec_model))
train_df['vector_sentence2'] = train_df['sentence2'].apply(lambda x: get_sentence_vector(x, word2vec_model))

train_df = train_df.dropna(subset=['vector_sentence1', 'vector_sentence2'])

pairs = []
labels = []
for _, row in train_df.iterrows():
    pairs.append([row['vector_sentence1'], row['vector_sentence2']])
    labels.append(row['label'])

pairs = np.array(pairs)
labels = np.array(labels)

input_shape = (100,)  

input_1 = Input(shape=input_shape)
input_2 = Input(shape=input_shape)

embedding_layer = Embedding(input_dim=1, output_dim=100, input_length=input_shape[0])
lstm_layer = LSTM(64)

encoded_1 = lstm_layer(embedding_layer(input_1))
encoded_2 = lstm_layer(embedding_layer(input_2))

distance = Lambda(lambda x: K.abs(x[0] - x[1]))([encoded_1, encoded_2])

output = Dense(1, activation='sigmoid')(distance)

siamese_model = Model(inputs=[input_1, input_2], outputs=output)

siamese_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

siamese_model.fit([pairs[:, 0], pairs[:, 1]], labels, epochs=10, batch_size=32, validation_split=0.2)

test_pairs = []
test_labels = []

for _, row in test_df.iterrows():
    vector1 = get_sentence_vector(row['sentence1'], word2vec_model)
    vector2 = get_sentence_vector(row['sentence2'], word2vec_model)

    if vector1 is not None and vector2 is not None:
        test_pairs.append([vector1, vector2])
        test_labels.append(row['label'])

test_pairs = np.array(test_pairs)
test_labels = np.array(test_labels)

predictions = siamese_model.predict([test_pairs[:, 0], test_pairs[:, 1]])

binary_predictions = (predictions > 0.5).astype(int)

accuracy = accuracy_score(test_labels, binary_predictions)
print(f'Siamese Model Accuracy on Test Set: {accuracy}')


Found cached dataset phrase_similarity (C:/Users/sanch/.cache/huggingface/datasets/PiC___phrase_similarity/PS-hard/2.0.1/7edf99614bd0c5125d3fc3b7d592122a988f23efc1e72dd1e01759bc68359302)


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/10
141/141 [==============================] - 29s 61ms/step - loss: 0.6932 - accuracy: 0.4955 - val_loss: 0.6932 - val_accuracy: 0.4906
Epoch 2/10
141/141 [==============================] - 6s 44ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.4906
Epoch 3/10
141/141 [==============================] - 6s 42ms/step - loss: 0.6932 - accuracy: 0.4993 - val_loss: 0.6932 - val_accuracy: 0.4906
Epoch 4/10
141/141 [==============================] - 6s 45ms/step - loss: 0.6932 - accuracy: 0.5022 - val_loss: 0.6933 - val_accuracy: 0.4906
Epoch 5/10
141/141 [==============================] - 6s 45ms/step - loss: 0.6932 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4906
Epoch 6/10
141/141 [==============================] - 6s 42ms/step - loss: 0.6932 - accuracy: 0.5054 - val_loss: 0.6932 - val_accuracy: 0.4906
Epoch 7/10
141/141 [==============================] - 6s 40ms/step - loss: 0.6932 - accuracy: 0.4922 - val_loss: 0.6933 - val_accuracy: 0.491

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from gensim.models import Word2Vec
from datasets import load_dataset

pic_dataset = load_dataset('PiC/phrase_similarity', 'PS-hard')

pic_df = pd.DataFrame(pic_dataset['train'])

label_encoder = LabelEncoder()
pic_df['label'] = label_encoder.fit_transform(pic_df['label'])

corpus = pic_df['sentence1'].tolist() + pic_df['sentence2'].tolist()
word2vec_model = Word2Vec(sentences=[sentence.split() for sentence in corpus], vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

pic_df['vector_sentence1'] = pic_df['sentence1'].apply(lambda x: get_sentence_vector(x, word2vec_model))
pic_df['vector_sentence2'] = pic_df['sentence2'].apply(lambda x: get_sentence_vector(x, word2vec_model))

pic_df = pic_df.dropna(subset=['vector_sentence1', 'vector_sentence2'])

X = np.array(list(zip(pic_df['vector_sentence1'], pic_df['vector_sentence2'])))
y = pic_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(64, input_shape=(2, 100))) 
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Found cached dataset phrase_similarity (C:/Users/sanch/.cache/huggingface/datasets/PiC___phrase_similarity/PS-hard/2.0.1/7edf99614bd0c5125d3fc3b7d592122a988f23efc1e72dd1e01759bc68359302)


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/10


C:\Users\sanch\anaconda3\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


141/141 [==============================] - 8s 22ms/step - loss: 0.6956 - accuracy: 0.5056 - val_loss: 0.6930 - val_accuracy: 0.5085
Epoch 2/10
141/141 [==============================] - 2s 12ms/step - loss: 0.6948 - accuracy: 0.4888 - val_loss: 0.6939 - val_accuracy: 0.4906
Epoch 3/10
141/141 [==============================] - 2s 13ms/step - loss: 0.6949 - accuracy: 0.4913 - val_loss: 0.6936 - val_accuracy: 0.4942
Epoch 4/10
141/141 [==============================] - 2s 12ms/step - loss: 0.6937 - accuracy: 0.5016 - val_loss: 0.6944 - val_accuracy: 0.4915
Epoch 5/10
141/141 [==============================] - 2s 12ms/step - loss: 0.6937 - accuracy: 0.4955 - val_loss: 0.6930 - val_accuracy: 0.5040
Epoch 6/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6934 - accuracy: 0.4897 - val_loss: 0.6931 - val_accuracy: 0.5049
Epoch 7/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6930 - accuracy: 0.5120 - val_loss: 0.6934 - val_accuracy: 0.5085
Epoch 8/10

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from gensim.models import Word2Vec
from datasets import load_dataset

pic_dataset = load_dataset('PiC/phrase_similarity', 'PS-hard')

pic_df = pd.DataFrame(pic_dataset['train'])
label_encoder = LabelEncoder()
pic_df['label'] = label_encoder.fit_transform(pic_df['label'])

corpus = pic_df['sentence1'].tolist() + pic_df['sentence2'].tolist()
word2vec_model = Word2Vec(sentences=[sentence.split() for sentence in corpus], vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

pic_df['vector_sentence1'] = pic_df['sentence1'].apply(lambda x: get_sentence_vector(x, word2vec_model))
pic_df['vector_sentence2'] = pic_df['sentence2'].apply(lambda x: get_sentence_vector(x, word2vec_model))

pic_df = pic_df.dropna(subset=['vector_sentence1', 'vector_sentence2'])

X = np.array(list(zip(pic_df['vector_sentence1'], pic_df['vector_sentence2'])))
y = pic_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(SimpleRNN(64, input_shape=(2, 100), activation='relu'))  # Adjust input_shape based on your sentence vector dimensions
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Found cached dataset phrase_similarity (C:/Users/sanch/.cache/huggingface/datasets/PiC___phrase_similarity/PS-hard/2.0.1/7edf99614bd0c5125d3fc3b7d592122a988f23efc1e72dd1e01759bc68359302)


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/10


C:\Users\sanch\anaconda3\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


141/141 [==============================] - 5s 16ms/step - loss: 0.7005 - accuracy: 0.4940 - val_loss: 0.6939 - val_accuracy: 0.5085
Epoch 2/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6955 - accuracy: 0.5042 - val_loss: 0.6939 - val_accuracy: 0.5085
Epoch 3/10
141/141 [==============================] - 2s 13ms/step - loss: 0.6962 - accuracy: 0.5013 - val_loss: 0.6946 - val_accuracy: 0.4915
Epoch 4/10
141/141 [==============================] - 1s 11ms/step - loss: 0.6938 - accuracy: 0.5011 - val_loss: 0.6970 - val_accuracy: 0.4915
Epoch 5/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6950 - accuracy: 0.4933 - val_loss: 0.6979 - val_accuracy: 0.4915
Epoch 6/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6942 - accuracy: 0.4926 - val_loss: 0.6933 - val_accuracy: 0.4942
Epoch 7/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6942 - accuracy: 0.4955 - val_loss: 0.6942 - val_accuracy: 0.4897
Epoch 8/10

In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from gensim.models import FastText
from datasets import load_dataset

pic_dataset = load_dataset('PiC/phrase_similarity', 'PS-hard')

pic_df = pd.DataFrame(pic_dataset['train'])

label_encoder = LabelEncoder()
pic_df['label'] = label_encoder.fit_transform(pic_df['label'])

corpus = pic_df['sentence1'].tolist() + pic_df['sentence2'].tolist()
fasttext_model = FastText(sentences=[sentence.split() for sentence in corpus], vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

pic_df['vector_sentence1'] = pic_df['sentence1'].apply(lambda x: get_sentence_vector(x, fasttext_model))
pic_df['vector_sentence2'] = pic_df['sentence2'].apply(lambda x: get_sentence_vector(x, fasttext_model))

pic_df = pic_df.dropna(subset=['vector_sentence1', 'vector_sentence2'])

X = np.array(list(zip(pic_df['vector_sentence1'], pic_df['vector_sentence2'])))
y = pic_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(SimpleRNN(64, input_shape=(2, 100), activation='relu'))  # Adjust input_shape based on your sentence vector dimensions
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Found cached dataset phrase_similarity (C:/Users/sanch/.cache/huggingface/datasets/PiC___phrase_similarity/PS-hard/2.0.1/7edf99614bd0c5125d3fc3b7d592122a988f23efc1e72dd1e01759bc68359302)


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/10


C:\Users\sanch\anaconda3\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


141/141 [==============================] - 5s 16ms/step - loss: 0.6995 - accuracy: 0.4949 - val_loss: 0.6950 - val_accuracy: 0.4915
Epoch 2/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6964 - accuracy: 0.4857 - val_loss: 0.6946 - val_accuracy: 0.5085
Epoch 3/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6959 - accuracy: 0.4996 - val_loss: 0.6941 - val_accuracy: 0.4915
Epoch 4/10
141/141 [==============================] - 1s 11ms/step - loss: 0.6957 - accuracy: 0.4924 - val_loss: 0.6978 - val_accuracy: 0.4915
Epoch 5/10
141/141 [==============================] - 1s 10ms/step - loss: 0.6968 - accuracy: 0.4893 - val_loss: 0.6950 - val_accuracy: 0.4915
Epoch 6/10
141/141 [==============================] - 1s 10ms/step - loss: 0.6943 - accuracy: 0.5011 - val_loss: 0.6937 - val_accuracy: 0.4915
Epoch 7/10
141/141 [==============================] - 1s 11ms/step - loss: 0.6939 - accuracy: 0.5011 - val_loss: 0.6933 - val_accuracy: 0.4924
Epoch 8/10

In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from gensim.models import Word2Vec
from datasets import load_dataset

pic_dataset = load_dataset('PiC/phrase_similarity', 'PS-hard')

pic_df = pd.DataFrame(pic_dataset['train'])

label_encoder = LabelEncoder()
pic_df['label'] = label_encoder.fit_transform(pic_df['label'])

corpus = pic_df['sentence1'].tolist() + pic_df['sentence2'].tolist()
corpus = pic_df['sentence1'].tolist() + pic_df['sentence2'].tolist()
fasttext_model = FastText(sentences=[sentence.split() for sentence in corpus], vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

pic_df['vector_sentence1'] = pic_df['sentence1'].apply(lambda x: get_sentence_vector(x, fasttext_model))
pic_df['vector_sentence2'] = pic_df['sentence2'].apply(lambda x: get_sentence_vector(x, fasttext_model))

pic_df = pic_df.dropna(subset=['vector_sentence1', 'vector_sentence2'])

X = np.array(list(zip(pic_df['vector_sentence1'], pic_df['vector_sentence2'])))
y = pic_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(SimpleRNN(64, input_shape=(2, 100), activation='relu'))  # Adjust input_shape based on your sentence vector dimensions
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Found cached dataset phrase_similarity (C:/Users/sanch/.cache/huggingface/datasets/PiC___phrase_similarity/PS-hard/2.0.1/7edf99614bd0c5125d3fc3b7d592122a988f23efc1e72dd1e01759bc68359302)


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/10


C:\Users\sanch\anaconda3\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


141/141 [==============================] - 5s 18ms/step - loss: 0.6967 - accuracy: 0.4996 - val_loss: 0.7021 - val_accuracy: 0.4915
Epoch 2/10
141/141 [==============================] - 1s 10ms/step - loss: 0.6958 - accuracy: 0.4906 - val_loss: 0.6930 - val_accuracy: 0.5076
Epoch 3/10
141/141 [==============================] - 1s 11ms/step - loss: 0.6943 - accuracy: 0.5042 - val_loss: 0.6970 - val_accuracy: 0.4915
Epoch 4/10
141/141 [==============================] - 1s 10ms/step - loss: 0.6938 - accuracy: 0.4913 - val_loss: 0.6935 - val_accuracy: 0.4915
Epoch 5/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6937 - accuracy: 0.4949 - val_loss: 0.6931 - val_accuracy: 0.5085
Epoch 6/10
141/141 [==============================] - 1s 11ms/step - loss: 0.6942 - accuracy: 0.4920 - val_loss: 0.6934 - val_accuracy: 0.4915
Epoch 7/10
141/141 [==============================] - 2s 11ms/step - loss: 0.6940 - accuracy: 0.4929 - val_loss: 0.6932 - val_accuracy: 0.4880
Epoch 8/10

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from gensim.models import Word2Vec
from datasets import load_dataset

pic_dataset = load_dataset('PiC/phrase_similarity', 'PS-hard')

pic_df = pd.DataFrame(pic_dataset['train'])

label_encoder = LabelEncoder()
pic_df['label'] = label_encoder.fit_transform(pic_df['label'])

corpus = pic_df['sentence1'].tolist() + pic_df['sentence2'].tolist()
fasttext_model = FastText(sentences=[sentence.split() for sentence in corpus], vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(sentence, model):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

pic_df['vector_sentence1'] = pic_df['sentence1'].apply(lambda x: get_sentence_vector(x, fasttext_model))
pic_df['vector_sentence2'] = pic_df['sentence2'].apply(lambda x: get_sentence_vector(x, fasttext_model))

pic_df = pic_df.dropna(subset=['vector_sentence1', 'vector_sentence2'])


X = np.array(list(zip(pic_df['vector_sentence1'], pic_df['vector_sentence2'])))
y = pic_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(64, input_shape=(2, 100)))  # Adjust input_shape based on your sentence vector dimensions
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


Found cached dataset phrase_similarity (C:/Users/sanch/.cache/huggingface/datasets/PiC___phrase_similarity/PS-hard/2.0.1/7edf99614bd0c5125d3fc3b7d592122a988f23efc1e72dd1e01759bc68359302)


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/20


C:\Users\sanch\anaconda3\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


141/141 [==============================] - 8s 20ms/step - loss: 0.6961 - accuracy: 0.5002 - val_loss: 0.6959 - val_accuracy: 0.5085
Epoch 2/20
141/141 [==============================] - 2s 12ms/step - loss: 0.6950 - accuracy: 0.4851 - val_loss: 0.6942 - val_accuracy: 0.5085
Epoch 3/20
141/141 [==============================] - 2s 11ms/step - loss: 0.6945 - accuracy: 0.4866 - val_loss: 0.6934 - val_accuracy: 0.4871
Epoch 4/20
141/141 [==============================] - 2s 11ms/step - loss: 0.6940 - accuracy: 0.4998 - val_loss: 0.6931 - val_accuracy: 0.4897
Epoch 5/20
141/141 [==============================] - 2s 11ms/step - loss: 0.6935 - accuracy: 0.5011 - val_loss: 0.6946 - val_accuracy: 0.4915
Epoch 6/20
141/141 [==============================] - 2s 11ms/step - loss: 0.6938 - accuracy: 0.4980 - val_loss: 0.6931 - val_accuracy: 0.5013
Epoch 7/20
141/141 [==============================] - 2s 11ms/step - loss: 0.6937 - accuracy: 0.4951 - val_loss: 0.6935 - val_accuracy: 0.4880
Epoch 8/20